<a href="https://colab.research.google.com/github/Taichuanlee/ShiftWizard/blob/main/%E6%8A%BD%E7%8F%AD%E5%88%A5_ver_github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio


<ipython-input-45-11cc1a33298d>:75: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if schedule.at[employee, month] == original[i]:
<ipython-input-45-11cc1a33298d>:99: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if schedule.at[top_employee, month] == original[i] and schedule.at[employee, month] != original[i]:


In [ ]:
pip install gspread oauth2client

<ipython-input-45-11cc1a33298d>:75: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if schedule.at[employee, month] == original[i]:
<ipython-input-45-11cc1a33298d>:99: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if schedule.at[top_employee, month] == original[i] and schedule.at[employee, month] != original[i]:


In [ ]:
!pip install xlsxwriter


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 3.2 MB/s eta 0:00:00


最終結果輸出xlsx

In [ ]:
import gradio as gr
import pandas as pd
import random
from io import BytesIO
import tempfile

# 定義檢查邏輯，確保班別需求滿足
def check_shift_needs(final_df, shift_needs, months):
    for month in months:
        count_A = (final_df[month] == 'A').sum()
        count_E = (final_df[month] == 'E').sum()
        count_N = (final_df[month] == 'N').sum()
        if count_A != shift_needs["A"] or count_E != shift_needs["E"] or count_N != shift_needs["N"]:
            return False
    return True

# Step 1: 定義班表分配邏輯
def assign_shifts_based_on_month_preferences(employees, shift_needs, months):
    employees = employees.set_index('name')  # 設定姓名為索引
    final_df = pd.DataFrame(index=employees.index, columns=months)

    while True:
        final_df[:] = None
        for i, month in enumerate(months):
            candidates_for_A = [emp for emp, choices in employees.iterrows() if choices.iloc[i] == "A"]
            candidates_for_E = [emp for emp, choices in employees.iterrows() if choices.iloc[i] == "E"]
            candidates_for_N = [emp for emp, choices in employees.iterrows() if choices.iloc[i] == "N"]

            selected_A = random.sample(candidates_for_A, min(len(candidates_for_A), shift_needs["A"]))
            selected_E = random.sample(candidates_for_E, min(len(candidates_for_E), shift_needs["E"]))
            selected_N = random.sample(candidates_for_N, min(len(candidates_for_N), shift_needs["N"]))

            remaining_A = shift_needs["A"] - len(selected_A)
            remaining_E = shift_needs["E"] - len(selected_E)
            remaining_N = shift_needs["N"] - len(selected_N)

            not_selected = set(employees.index) - set(selected_A) - set(selected_E) - set(selected_N)

            if remaining_A > 0:
                selected_A += random.sample(list(not_selected), remaining_A)
                not_selected -= set(selected_A)

            if remaining_E > 0:
                selected_E += random.sample(list(not_selected), remaining_E)
                not_selected -= set(selected_E)

            if remaining_N > 0:
                selected_N += random.sample(list(not_selected), remaining_N)

            for emp in selected_A:
                final_df.at[emp, month] = 'A'
            for emp in selected_E:
                final_df.at[emp, month] = 'E'
            for emp in selected_N:
                final_df.at[emp, month] = 'N'

        for emp in employees.index:
            if 'A' not in final_df.loc[emp].values:
                available_months = [month for month in months if final_df.at[emp, month] != 'A']
                if available_months:
                    chosen_month = random.choice(available_months)
                    final_df.at[emp, chosen_month] = 'A'

        if check_shift_needs(final_df, shift_needs, months):
            break

    final_df.reset_index(inplace=True)  # 確保 "Name" 在結果中顯示
    final_df.rename(columns={"index": "Name"}, inplace=True)

    return final_df

# Step 2: 保底機制
def ensure_minimum_matches_with_swap(schedule, original_preferences, months, min_matches=2):
    def calculate_matches(employee, original, schedule):
        matches = 0
        for i, month in enumerate(months):
            if schedule.loc[employee, month] == original.iloc[i]:
                matches += 1
        return matches

    has_suai_ghosts = True

    while has_suai_ghosts:
        has_suai_ghosts = False  # 假設沒有衰鬼

        for employee, original in original_preferences.iterrows():
            matches = calculate_matches(employee, original, schedule)

            if matches < min_matches:
                has_suai_ghosts = True

                for top_employee, top_original in original_preferences.iterrows():
                    if top_employee == employee:
                        continue

                    top_matches = calculate_matches(top_employee, top_original, schedule)

                    if top_matches > matches:
                        for i, month in enumerate(months):
                            if schedule.loc[top_employee, month] == original.iloc[i] and schedule.loc[employee, month] != original.iloc[i]:
                                schedule.loc[employee, month], schedule.loc[top_employee, month] = schedule.loc[top_employee, month], schedule.loc[employee, month]
                                matches = calculate_matches(employee, original, schedule)
                                if matches >= min_matches:
                                    break
                        if matches >= min_matches:
                            break
    return schedule

# Step 3: 依據生成的班表隨機抽取員工並替換班別
def assign_special_shifts(schedule_df, months):
    updated_schedule = schedule_df.copy()
    selected_employees = set()

    # Step 1: 處理 N 班，逐月隨機選取一位員工替換成 N1
    for month in months:
        available_employees = [emp for emp in schedule_df.index if schedule_df.at[emp, month] == 'N' and emp not in selected_employees]
        if available_employees:
            selected_N = random.choice(available_employees)
            updated_schedule.at[selected_N, month] = 'N1'
            selected_employees.add(selected_N)

    # Step 2: 處理 E 班，逐月隨機選取一位員工替換成 E1
    for month in months:
        available_employees = [emp for emp in schedule_df.index if schedule_df.at[emp, month] == 'E' and emp not in selected_employees]
        if available_employees:
            selected_E = random.choice(available_employees)
            updated_schedule.at[selected_E, month] = 'E1'
            selected_employees.add(selected_E)

    # Step 3: 處理 A 班，逐月隨機選取兩位員工替換成 A1 和 A2
    for month in months:
        available_employees = [emp for emp in schedule_df.index if schedule_df.at[emp, month] == 'A' and emp not in selected_employees]
        if len(available_employees) >= 2:
            selected_A1, selected_A2 = random.sample(available_employees, 2)
            updated_schedule.at[selected_A1, month] = 'A1'
            updated_schedule.at[selected_A2, month] = 'A2'
            selected_employees.add(selected_A1)
            selected_employees.add(selected_A2)

    return updated_schedule

# 定義主要的處理函數
def process_schedule(uploaded_file, a_count, e_count, n_count, min_matches):
    employees = pd.read_csv(uploaded_file)
    shift_needs = {"A": a_count, "E": e_count, "N": n_count}
    months = ["January", "February", "March", "April", "May", "June"]

    # Step 1: 生成初步班表
    schedule_df = assign_shifts_based_on_month_preferences(employees, shift_needs, months)

    # Step 2: 執行保底機制
    second_schedule_df = ensure_minimum_matches_with_swap(schedule_df.copy(), employees, months, min_matches=min_matches)

    # Step 3: 特殊班別分配
    final_schedule_df = assign_special_shifts(second_schedule_df, months)

    # 將結果保存為 Excel 文件
    with tempfile.NamedTemporaryFile(delete=False, suffix=".xlsx") as tmp:
        with pd.ExcelWriter(tmp.name, engine='xlsxwriter') as writer:
            final_schedule_df.to_excel(writer, sheet_name='Schedule', index=False)
        return final_schedule_df, tmp.name

# 定義 Gradio 介面
inputs = [
    gr.File(label="上傳員工選擇文件"),
    gr.Slider(minimum=1, maximum=50, value=5, label="A 班人數", step=1),
    gr.Slider(minimum=1, maximum=50, value=3, label="E 班人數", step=1),
    gr.Slider(minimum=1, maximum=50, value=2, label="N 班人數", step=1),
    gr.Slider(minimum=1, maximum=6, value=2, label="每人最少的班別匹配數", step=1)
]

outputs = [
    gr.DataFrame(label="最終分配結果"),
    gr.File(label="下載班表")
]

# 啟動 Gradio 應用
gr.Interface(fn=process_schedule, inputs=inputs, outputs=outputs, title="班表分配系統（含保底機制與特殊班別）").launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6d4fa0c162b8faf013.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
